Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.  
Copyright (c) 2017, Pytorch contributors All rights reserved.
## BSD 3-Clause License
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Model Migration from GPU to the Intel&reg; Gaudi&reg; 2 AI Processor 

The GPU Migration toolkit simplifies migrating PyTorch models that run on GPU-based architecture to run on the Intel® Gaudi® AI accelerator. Rather than manually replacing Python API calls that have dependencies on GPU libraries with Gaudi-specific API calls, the toolkit automates this process so you can run your model with fewer modifications.  

In this notebook we will demonstrate how to use the GPU Migration toolset on a ResNet50 model which is based on open source implementation of ResNet50.  

Refer to the [GPU Migration Toolkit](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html) for more information.  

In addtion to this ResNet50 migration, there are addtional GPU Migration example on the Intel Gaudi GitHub page [here](https://github.com/HabanaAI/Model-References/tree/master/PyTorch/examples/gpu_migration)

#### Clone the Model-References repository

In [1]:
%cd ~/Gaudi-tutorials/PyTorch/GPU_Migration
!git clone -b 1.16.2 https://github.com/habanaai/Model-References

/root/Gaudi-tutorials/PyTorch/GPU_Migration
Cloning into 'Model-References'...
remote: Enumerating objects: 18994, done.
remote: Counting objects: 100% (4783/4783), done.
remote: Compressing objects: 100% (2089/2089), done.
remote: Total 18994 (delta 2579), reused 4522 (delta 2380), pack-reused 14211 (from 1)
Receiving objects: 100% (18994/18994), 119.29 MiB | 40.87 MiB/s, done.
Resolving deltas: 100% (10265/10265), done.
Updating files: 100% (1622/1622), done.


#### Download dataset (Optional)
To fully run this example you can download [Tiny ImageNet dataset](https://www.kaggle.com/datasets/saumandas/tiny-imagenet) from Kaggle. It needs to be organized according to PyTorch requirements, and as specified in the scripts of [imagenet-multiGPU.torch](https://github.com/soumith/imagenet-multiGPU.torch).   
Run the cells in this section to continue downloading the dataset to your local folder.

> **Note: You do not need to have the dataset loaded to see the Migration steps and logging.**

Configure your Kaggle API credentials. **You'll need to download your Kaggle API token (kaggle.json) from your Kaggle account settings to the local folder first.** 
Running this cell will place this file in the ~/.kaggle/ directory on this system.

In [2]:
!pip install --quiet kaggle
import os
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
!cp ./kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d saumandas/tiny-imagenet
!chmod 600 ./tiny-imagenet.zip
os.makedirs("/root/datasets/", exist_ok=True)
!unzip ./tiny-imagenet.zip  -x "tiny-imagenet-200/test*" -d /root/datasets/

#### Navigate to the model example to begin the run

In [3]:
%cd ~/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision

/root/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Import GPU Migration Toolkit package and Habana Torch Library
Look into train.py, you will see in the first line that we will load the `gpu.migration` libary which is already inclucded in the Intel Gaudi Software: 

In [4]:
%%sh
cat -n train.py | head -n 20 | tail -n 17

     4	import habana_frameworks.torch.gpu_migration
     5	import datetime
     6	import os
     7	import time
     8	import warnings
     9	
    10	import presets
    11	import torch
    12	import torch.utils.data
    13	import torchvision
    14	import transforms
    15	import utils
    16	import habana_frameworks.torch.utils.experimental as htexp
    17	from sampler import RASampler
    18	from torch import nn
    19	from torch.utils.data.dataloader import default_collate
    20	from torchvision.transforms.functional import InterpolationMode


#### Placing mark_step()
You will have to place the mark_step() function after the optimizer and loss.backward calculations

In [5]:
%%sh
cat -n train.py | head -n 51 | tail -n 14

    38	        optimizer.zero_grad(set_to_none=True)
    39	        if scaler is not None:
    40	            scaler.scale(loss).backward()
    41	            htcore.mark_step()
    42	            if args.clip_grad_norm is not None:
    43	                # we should unscale the gradients of optimizer's assigned params if do gradient clipping
    44	                scaler.unscale_(optimizer)
    45	                nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
    46	            scaler.step(optimizer)
    47	            htcore.mark_step()
    48	            scaler.update()
    49	        else:
    50	            loss.backward()
    51	            htcore.mark_step()


#### Run the following command to start multi-HPU training.
We're now ready to run the training.  You will see that we've added the logging command at the beginning of the run `GPU_MIGRATION_LOG_LEVEL=1` to show the output.   There are no other changes to the run command are needed.   As you see the training run is started, you will see the log files show exactly where the code changes are happening to change from GPU to Intel Gaudi, including the file name and location.

```bash
GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path="/root/datasets/tiny-imagenet-200/" --workers=8 --epochs=1 --opt=sgd --amp
```

In [6]:
!GPU_MIGRATION_LOG_LEVEL=1 torchrun --nproc_per_node 1 train.py --batch-size=256 --model=resnet50 --device=cuda --data-path="/root/datasets/tiny-imagenet-200/" --workers=8 --epochs=1 --opt=sgd --amp

/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/gpu_migration/__init__.py:46: UserWarning: apex not installed, gpu_migration will not swap api for this package.
  warnings.warn(
gpu migration log will be saved to /var/log/habana_logs/gpu_migration_logs/2024-09-05/17-34-16/gpu_migration_770.log
[2024-09-05 17:34:17] /root/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/torchvision/utils.py:265
    [context]:     torch.cuda.set_device(args.gpu)

    [hpu_match]: torch.cuda.set_device(device=0, ) --> torch.hpu.set_device(hpu:0)

| distributed init (rank 0): env://
[2024-09-05 17:34:17] /root/Gaudi-tutorials/PyTorch/GPU_Migration/Model-References/PyTorch/examples/gpu_migration/computer_vision/classification/

In [7]:
exit()